# Análise Exploratória de Dados - Missing Data - Censo Escolar 2024

## Ligação com BD SQLite + carga de Dataframe desde o dataset

In [3]:
import pandas as pd
import sqlite3

# =================================================================
# FASE 1: CONFIGURAÇÃO DOS CAMINHOS
# =================================================================

# Caminho para o arquivo RAW original
RAW_CSV_PATH = "/home/fcsr/Documentos/Dbeaver/qualidade_dados_censo_escolar_2024/data/1_raw/microdados_ed_basica_2024.csv"

# Caminho para o banco de dados SQLite
DB_PATH = "/home/fcsr/Documentos/Dbeaver/qualidade_dados_censo_escolar_2024/data/0_sqlite/censo_escolar"

# Nomes das tabelas no SQLite
TABLE_TRUSTED = 'microdados_ed_basica_trusted'
TABLE_DICIONARIO = 'dicionario'


# =================================================================
# FASE 2: CARGA DOS DADOS E EXTRAÇÃO DAS CHAVES
# =================================================================

# --- Carregar dados do SQLite (Trusted e Dicionário) ---
conn = sqlite3.connect(DB_PATH)
df_trusted = pd.read_sql_query(f"SELECT * FROM {TABLE_TRUSTED}", conn)
df_dicionario = pd.read_sql_query(f"SELECT * FROM {TABLE_DICIONARIO}", conn)
conn.close()

# --- Carregar apenas o cabeçalho do arquivo RAW para pegar as colunas ---
# nrows=0 é um truque para ler apenas o cabeçalho, sem carregar dados, o que é muito rápido.
df_raw = pd.read_csv(RAW_CSV_PATH, sep=';', nrows=0, low_memory=False, encoding='latin1')

# --- Extrair as listas de colunas/variáveis como conjuntos (sets) para fácil comparação ---
colunas_raw = set(df_raw.columns)
colunas_trusted = set(df_trusted.columns)
variaveis_dicionario = set(df_dicionario['VARIAVEL'])

print(f"Total de colunas na RAW: {len(colunas_raw)}")
print(f"Total de colunas na TRUSTED: {len(colunas_trusted)}")
print(f"Total de variáveis no DICIONÁRIO: {len(df_dicionario['VARIAVEL'])}")
print(f"Total de variáveis ÚNICAS no DICIONÁRIO: {len(variaveis_dicionario)}")
print("-" * 50)


# =================================================================
# FASE 3: DIAGNÓSTICO E COMPARAÇÃO
# =================================================================

# --- 1. Análise de Duplicatas no Dicionário ---
if len(df_dicionario['VARIAVEL']) != len(variaveis_dicionario):
    print("\n🚨 ATENÇÃO: Foram encontradas variáveis duplicadas no Dicionário!")
    duplicatas = df_dicionario[df_dicionario['VARIAVEL'].duplicated(keep=False)].sort_values('VARIAVEL')
    print("Variáveis duplicadas:")
    display(duplicatas)
else:
    print("\n✅ SUCESSO: Nenhuma variável duplicada encontrada no Dicionário.")

print("-" * 50)

# --- 2. Comparação: Dicionário vs. Trusted ---
# Quais variáveis estão no dicionário mas não na tabela trusted?
# Estas são as colunas "perdidas" durante o ETL.
colunas_perdidas = sorted(list(variaveis_dicionario - colunas_trusted))

if colunas_perdidas:
    print(f"\n🚨 ATENÇÃO: {len(colunas_perdidas)} colunas estão no Dicionário mas FALTAM na camada Trusted:")
    print(colunas_perdidas)
else:
    print("\n✅ SUCESSO: Todas as variáveis do Dicionário estão presentes na camada Trusted.")

print("-" * 50)

# --- 3. Comparação Extra: Trusted vs. Dicionário ---
# Existem colunas na trusted que não têm definição no dicionário?
colunas_sem_definicao = sorted(list(colunas_trusted - variaveis_dicionario))

if colunas_sem_definicao:
    print(f"\n🚨 ATENÇÃO: {len(colunas_sem_definicao)} colunas estão na Trusted mas FALTAM no Dicionário:")
    print(colunas_sem_definicao)
else:
    print("\n✅ SUCESSO: Todas as colunas da camada Trusted possuem uma definição no Dicionário.")

Total de colunas na RAW: 426
Total de colunas na TRUSTED: 426
Total de variáveis no DICIONÁRIO: 476
Total de variáveis ÚNICAS no DICIONÁRIO: 476
--------------------------------------------------

✅ SUCESSO: Nenhuma variável duplicada encontrada no Dicionário.
--------------------------------------------------

🚨 ATENÇÃO: 50 colunas estão no Dicionário mas FALTAM na camada Trusted:
['IN_AGUA_FILTRADA', 'IN_BANHEIRO_DENTRO_PREDIO', 'IN_BANHEIRO_FORA_PREDIO', 'IN_BAS', 'IN_BERCARIO', 'IN_BRASIL_ALFABETIZADO', 'IN_CONVENIADA_PP', 'IN_DEPENDENCIAS_PNE', 'IN_ENERGIA_GERADOR', 'IN_ENERGIA_OUTROS', 'IN_EQUIP_FAX', 'IN_EQUIP_FOTO', 'IN_EQUIP_RETROPROJETOR', 'IN_EQUIP_VIDEOCASSETE', 'IN_FINAL_SEMANA', 'IN_FORMACAO_ALTERNANCIA', 'IN_FUNDAMENTAL_CICLOS', 'IN_GRUPOS_NAO_SERIADOS', 'IN_LAVANDERIA', 'IN_LIXO_JOGA_OUTRA_AREA', 'IN_LIXO_OUTROS', 'IN_LIXO_RECICLA', 'IN_LOCAL_FUNC_CASA_PROFESSOR', 'IN_LOCAL_FUNC_SALAS_EMPRESA', 'IN_LOCAL_FUNC_TEMPLO_IGREJA', 'IN_MATERIAL_ESP_INDIGENA', 'IN_MATERIAL_ESP_

#### Conclusão de bloco:
- Ligação com SQLite, ok!
- Número de colunas de RAW e TRUSTED checadas, ok!
- As colunas de trusted constam no dicionário, ok!
- As 50 variáveis mencionadas como não localizadas são aquelas que foram coletadas apenas até o Censo Escolar de 2023 

## Exploração de Missing Data

In [ ]:
import pandas as pd
import numpy as np

# =================================================================
# FASE 0: PRÉ-REQUISITO
# =================================================================
# Este script assume que os DataFrames 'df_trusted' e 'df_dicionario'
# já foram carregados na memória a partir do seu banco de dados SQLite.
# Se precisar carregá-los novamente, use o script anterior.

# Verificação para garantir que os DFs existem
if 'df_trusted' not in locals() or 'df_dicionario' not in locals():
    print("ERRO: Os DataFrames 'df_trusted' e 'df_dicionario' não foram encontrados.")
    print("Por favor, execute o script de conexão com o SQLite primeiro.")
else:
    print("DataFrames encontrados. Iniciando a análise agregada...")


    # =================================================================
    # FASE 1: UNPIVOT (MELT) DA TABELA TRUSTED
    # =================================================================
    # Transformamos a tabela de formato 'largo' para 'longo'.
    # Cada linha agora representará uma única célula da tabela original.
    print("Reorganizando a tabela de dados (melt)...")
    df_long = df_trusted.melt(var_name='VARIAVEL', value_name='VALOR')


    # =================================================================
    # FASE 2: JOIN COM O DICIONÁRIO
    # =================================================================
    # Juntamos a tabela longa com as informações de categoria do dicionário.
    print("Juntando dados com as categorias do dicionário...")
    df_dict_subset = df_dicionario[['VARIAVEL', 'GRUPO_CAT', 'SUB_GRUPO_CAT']].drop_duplicates()
    df_merged = pd.merge(df_long, df_dict_subset, on='VARIAVEL', how='left')
    
    # Preenche categorias para variáveis que não foram encontradas no dicionário
    df_merged['GRUPO_CAT'] = df_merged['GRUPO_CAT'].fillna('Sem Grupo')
    df_merged['SUB_GRUPO_CAT'] = df_merged['SUB_GRUPO_CAT'].fillna('Sem Subgrupo')


    # =================================================================
    # FASE 3: AGRUPAMENTO E CÁLCULO DAS MÉTRICAS
    # =================================================================
    print("Agrupando e calculando as métricas de qualidade...")
    
    # Agrupamos por categoria e agregamos os resultados
    # 'size' conta o total de células no grupo (incluindo nulos)
    # As lambdas contam os nulos e os placeholders (-100)
    df_grouped = df_merged.groupby(['GRUPO_CAT', 'SUB_GRUPO_CAT']).agg(
        total_celulas=('VALOR', 'size'),
        contagem_nulos=('VALOR', lambda x: x.isnull().sum()),
        contagem_placeholder=('VALOR', lambda x: (x == -100).sum())
    )

    # Cálculo das porcentagens
    total_missing = df_grouped['contagem_nulos'] + df_grouped['contagem_placeholder']
    df_grouped['%_de_Missing_Data'] = (total_missing / df_grouped['total_celulas']) * 100
    df_grouped['%_Preenchido'] = 100 - df_grouped['%_de_Missing_Data']
    
    # Adicionando o total de registros (número de linhas da tabela original)
    df_grouped['Total_de_Registros'] = len(df_trusted)


    # =================================================================
    # FASE 4: FORMATAÇÃO E APRESENTAÇÃO FINAL
    # =================================================================
    print("Formatando o relatório final...")

    # Selecionando e renomeando as colunas para o resultado final
    df_resultado = df_grouped[[
        'Total_de_Registros',
        '%_de_Missing_Data',
        '%_Preenchido'
    ]].copy()

    # Ordenando pelo % de Missing Data, do maior para o menor
    df_resultado = df_resultado.sort_values(by='%_de_Missing_Data', ascending=False)
    
    # Formatando as porcentagens para melhor visualização
    df_resultado['%_de_Missing_Data'] = df_resultado['%_de_Missing_Data'].map('{:.2f}%'.format)
    df_resultado['%_Preenchido'] = df_resultado['%_Preenchido'].map('{:.2f}%'.format)
    
    # Trazendo os grupos de volta como colunas
    df_resultado = df_resultado.reset_index()

    print("\n--- Relatório de Qualidade de Dados por Categoria ---")
    display(df_resultado)

DataFrames encontrados. Iniciando a análise agregada...
Reorganizando a tabela de dados (melt)...
Juntando dados com as categorias do dicionário...
Agrupando e calculando as métricas de qualidade...
